# Synchronizing

## _thread

In [ ]:
import _thread

a_lock = _thread.allocate_lock()

with a_lock:
    print("a_lock is locked while this executes")

## threading

### Lock

In [ ]:
lock.acquire()
try:
    ... access shared resource
finally:
    lock.release() # release lock, no matter wha

In [ ]:
if not lock.acquire(False):
    ... failed to lock the resource
else:
    try:
        ... access shared resource
    finally:
        lock.release()

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 24 12:06:02 2020

@author: Kévin
"""
import threading
             
class Counter(object):
    def __init__(self, start = 0):
        self.lock = threading.Lock()
        self.value = start
    def increment(self):
        print('Waiting for a lock')
        self.lock.acquire()
        try:
            print('Acquired a lock')
            self.value = self.value + 1
        finally:
            print('Released a lock')
            self.lock.release()

def worker(c):
    c.increment()
    print('Done')

if __name__ == '__main__':
    my_list=[]
    counter = Counter()
    for i in range(2):
        t = threading.Thread(target=worker, args=(counter,))
        my_list.append(t)
        t.start()

    print('Waiting for worker threads')
    main_thread = threading.currentThread()
    for t in my_list:
        t.join()
    print('Counter:'+ str(counter.value))

In [ ]:
lock = threading.Lock()

def get_first_part():
    lock.acquire()
    try:
        ... fetch data for first part from shared object
    finally:
        lock.release()
    return data

def get_second_part():
    lock.acquire()
    try:
        ... fetch data for second part from shared object
    finally:
        lock.release()
    return data

def get_both_parts():
    lock.acquire()
    try:
        first = get_first_part()
        second = get_second_part()
    finally:
        lock.release()
    return first, second

### ReLock

In [ ]:
lock = threading.Lock()
lock.acquire()
lock.acquire() # this will block

lock = threading.RLock()
lock.acquire()
lock.acquire() # this won't block

### Semaphore

In [ ]:
# -*- coding: utf-8 -*-
import threading
             
class Counter(object):
    def __init__(self, start = 0):
        self.semaphore = threading.BoundedSemaphore(value=1)
        self.value = start
    def increment(self):
        print('Waiting for a lock')
        self.semaphore.acquire()
        try:
            print('Acquired a lock')
            self.value = self.value + 1
        finally:
            print('Released a lock')
            self.semaphore.release()

def worker(c):
    c.increment()
    print('Done')

if __name__ == '__main__':
    my_list=[]
    counter = Counter()
    for i in range(2):
        t = threading.Thread(target=worker, args=(counter,))
        my_list.append(t)
        t.start()

    print('Waiting for worker threads')
    main_thread = threading.currentThread()
    for t in my_list:
        t.join()
    print('Counter:'+ str(counter.value))

## multiprocessing

In [ ]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print(q.get())    # prints "[42, None, 'hello']"
    p.join()

In [ ]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

In [ ]:
# -*- coding: utf-8 -*-
import multiprocessing 

# function to deposit to account 
def increment(variable):     
    variable.value = variable.value + 1

if __name__ == "__main__": 
    counter = multiprocessing.Value('i', 0)

    p1 = multiprocessing.Process(target=increment, args=(counter,)) 
    p2 = multiprocessing.Process(target=increment, args=(counter,)) 
  
    p1.start() 
    p2.start() 

    p1.join();
    p2.join();

    print("Final balance = "+str(counter.value)) 

### Lock

In [ ]:
import multiprocessing 
  
# function to withdraw from account 
def withdraw(balance, lock):     
    for _ in range(10000): 
        lock.acquire() 
        balance.value = balance.value - 1
        lock.release() 
  
# function to deposit to account 
def deposit(balance, lock):     
    for _ in range(10000): 
        lock.acquire() 
        balance.value = balance.value + 1
        lock.release() 
  
def perform_transactions(): 
  
    # initial balance (in shared memory) 
    balance = multiprocessing.Value('i', 100) 
  
    # creating a lock object 
    lock = multiprocessing.Lock() 
  
    # creating new processes 
    p1 = multiprocessing.Process(target=withdraw, args=(balance,lock)) 
    p2 = multiprocessing.Process(target=deposit, args=(balance,lock)) 
  
    # starting processes 
    p1.start() 
    p2.start() 
  
    # wait until processes are finished 
    p1.join() 
    p2.join() 
  
    # print final balance 
    print("Final balance = "+str(balance.value)) 
  
if __name__ == "__main__": 
    for _ in range(10): 
  
        # perform same transaction process 10 times 
        perform_transactions()

### RLock

### Semaphore